In [1]:
import os, json, copy, re, zipfile
from collections import OrderedDict
from damd_multiwoz.ontology import all_domains
import pprint
import random
import sqlite3
import random
import torch

/home/fanghongming/anaconda3/envs/CASPI/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 2023.10.7 debug damd_multiwoz/data_analysis.py 
# to 1. get action statistics for all domain/slots; 2. apply data augmentation 
data_path = 'damd_multiwoz/data/multi-woz/'
save_path = 'damd_multiwoz/data/multi-woz-analysis/'
save_path_exp = 'damd_multiwoz/data/multi-woz-processed/'
data_file = 'data.json'
attraction_db = 'damd_multiwoz/db/attraction-dbase.db'
pre_db_paths = {
            'attraction': 'damd_multiwoz/db/attraction_db_processed.json',
            'hospital': 'damd_multiwoz/db/hospital_db_processed.json',
            'hotel': 'damd_multiwoz/db/hotel_db_processed.json',
            'police': 'damd_multiwoz/db/police_db_processed.json',
            'restaurant': 'damd_multiwoz/db/restaurant_db_processed.json',
            'taxi': 'damd_multiwoz/db/taxi_db_processed.json',
            'train': 'damd_multiwoz/db/train_db_processed.json',
        }
device = 'cuda:0'

In [3]:
archive = zipfile.ZipFile(data_path+data_file+'.zip', 'r')

data = archive.open(data_file, 'r').read().decode('utf-8').lower()
#data = os.path.join(data_path, 'data.json')
ref_nos = list(set(re.findall(r'\"reference\"\: \"(\w+)\"', data)))
data = json.loads(data)

In [14]:
print(list(data.keys())[325])
print(list(data.keys())[326])

sng0732.json
pmul2201.json


In [4]:
pprint.pprint(data['sng01856.json']['goal'])            

{'attraction': {},
 'hospital': {},
 'hotel': {'book': {'day': 'tuesday',
                    'invalid': False,
                    'people': '6',
                    'pre_invalid': True,
                    'stay': '2'},
           'fail_book': {'stay': '3'},
           'fail_info': {},
           'info': {'internet': 'yes',
                    'parking': 'yes',
                    'pricerange': 'cheap',
                    'type': 'hotel'}},
 'message': ["you are looking for a <span class='emphasis'>place to "
             'stay</span>. the hotel should be in the <span '
             "class='emphasis'>cheap</span> price range and should be in the "
             "type of <span class='emphasis'>hotel</span>",
             "the hotel should <span class='emphasis'>include free "
             "parking</span> and should <span class='emphasis'>include free "
             'wifi</span>',
             "once you find the <span class='emphasis'>hotel</span> you want "
             "to book it fo

In [ ]:
pprint.pprint(data['sng01856.json']['log'])

In [54]:
pprint.pprint(data['sng01856.json']['goal'].keys())

dict_keys(['taxi', 'police', 'hospital', 'hotel', 'topic', 'attraction', 'train', 'message', 'restaurant'])


In [5]:
def fail_into_count(data):
    fail_dict = {}
    for fn, dial in data.items():
        goals = dial['goal']
        # get all the failable dials
        for domain in all_domains:
            if goals[domain].get('fail_book', False) or goals[domain].get('fail_info', False):
                fail_dict[fn] = {domain:goals[domain]}
    return fail_dict

def print_dialogue_text(data, fn):
    current_turn = 0
    for turn in data[fn]['log']:
        turn_text = turn['text']
        print(f'Turn {current_turn}: {turn_text}')
        current_turn += 1




In [78]:
fail_dict = fail_into_count(data)
random_key = random.sample(list(fail_dict.keys()), 10)



In [82]:
for i in range(10):
    print(random_key[i])
    pprint.pprint(fail_dict[random_key[i]])

pmul1210.json
{'attraction': {'fail_info': {'area': 'south', 'type': 'swimmingpool'},
                'info': {'area': 'south', 'type': 'cinema'},
                'reqt': ['entrance fee']}}
pmul3437.json
{'attraction': {'fail_info': {'type': 'nightclub'},
                'info': {'type': 'museum'},
                'reqt': ['phone']}}
pmul4671.json
{'hotel': {'book': {'day': 'thursday',
                    'invalid': False,
                    'people': '2',
                    'stay': '4'},
           'fail_book': {},
           'fail_info': {'area': 'north',
                         'parking': 'yes',
                         'pricerange': 'expensive',
                         'stars': '3'},
           'info': {'area': 'centre',
                    'parking': 'yes',
                    'pricerange': 'expensive',
                    'stars': '3'}}}
pmul1656.json
{'hotel': {'fail_info': {'area': 'east',
                         'internet': 'yes',
                         'parking': 'yes'

In [15]:
#pmul3437.json
''' 
{'attraction': {'fail_info': {'type': 'nightclub'},
                'info': {'type': 'museum'},
                'reqt': ['phone']}}
'''
print_dialogue_text(data, 'mul2423.json')
pprint.pprint(data['mul2423.json']['goal'])
pprint.pprint(data['mul2423.json']['log'])

Turn 0: yes, i am looking for a place to dine. i would prefer the city centre and am looking for halal food.
Turn 1: i am sorry i can find no place that serves that in the centre area. would you like to try another area or food type?
Turn 2: ok. is there an indian restaurant in the centre area. 
Turn 3: there are five indian restaurants in that area, what is your price-range? 
Turn 4: i don't have a price range. can you just recommend one of the five restaurants and tell me the postcode and the price range
Turn 5: i apologize. there were 9 differrent indian restaurants in the centre of town. my recommendation would be panahar. they are expensive. their postcode is cb12lf. anything else i can help with?
Turn 6: what is the post code for the expensive restaurant panahar?
Turn 7: the postcode is cb12lf.  is there anything else i can do for you?
Turn 8: i'd like some information on theatre's in town.
Turn 9: there are five theatres in town, are you looking for something in the center of to

In [18]:
pprint.pprint(data['mul2423.json']['goal'])

{'attraction': {'fail_info': {'type': 'concerthall'},
                'info': {'type': 'theatre'},
                'reqt': ['postcode', 'phone']},
 'hospital': {},
 'hotel': {},
 'message': ['you are traveling to cambridge and excited about seeing local '
             'tourist attractions',
             "you are looking for a <span class='emphasis'>place to "
             'dine</span>. the restaurant should be in the <span '
             "class='emphasis'>centre</span> and should serve <span "
             "class='emphasis'>halal</span> food",
             'if there is no such restaurant, how about one that serves <span '
             "class='emphasis'>indian</span> food",
             "make sure you get <span class='emphasis'>postcode</span> and "
             "<span class='emphasis'>price range</span>",
             "you are also looking for <span class='emphasis'>places to "
             'go</span> in town. the attraction should be in the type of <span '
             "class='emphasi

In [24]:
# query attraction-dbase.db, check if equal to attraction_db.json

# OK 是一样的

# 连接到数据库
conn = sqlite3.connect(attraction_db)  # 替换 'your_database.db' 为你的 .db 文件的路径
cursor = conn.cursor()

# 执行 SQL 查询
cursor.execute('SELECT * FROM attraction')  # 替换 'table_name' 为你想查询的表名
rows = cursor.fetchall()

# 打印查询结果
print(len(rows))
print(rows[:10])

# 关闭数据库连接
cursor.close()
conn.close()

79
[('1', 'pool way, whitehill road, off newmarket road', 'east', '?', 'abbey pool and astroturf pitch', '01223902088', 'cb58nt', '?', '?', 'swimmingpool'), ('2', 'park street', 'centre', '?', 'adc theatre', '01223300085', 'cb58as', '?', '?', 'theatre'), ('3', 'jesus lane', 'centre', 'free', 'all saints church', '01223452587', 'cb58bs', 'free', 'it opens normally daily from 9 a.m. to 5 p.m., at other times the key can be borrowed from a nearby key holder.', 'architecture'), ('4', 'heidelberg gardens, lion yard', 'centre', '5 pounds', 'ballare', '01223364222', 'cb23na', 'moderate', 'it opens from 10:30 p.m. to 03:30 a.m on thursday, from 11:00 p.m. to 04:00 a.m. on friday, from 10:00 p.m. to 03:30 a.m. on saturday, and from 10:00 p.m. to 02:30 a.m. on monday', 'nightclub'), ('5', '98 king street', 'centre', 'free', 'broughton house gallery', '01223314960', 'cb11ln', 'free', '?', 'museum'), ('6', "14 king's parade", 'south', 'free', 'byard art', '01223464646', 'cb21sj', 'free', 'it opens

In [7]:
# 统计每个 domain 的 条目数量
attraction_json = 'damd_multiwoz/db/attraction_db.json'
bus_json = 'damd_multiwoz/db/bus_db.json'
hospital_json = 'damd_multiwoz/db/hospital_db.json'
hotel_json = 'damd_multiwoz/db/hotel_db.json'
police_json = 'damd_multiwoz/db/police_db.json'
restaurant_json = 'damd_multiwoz/db/restaurant_db.json'
taxi_json = 'damd_multiwoz/db/taxi_db.json'
train_json = 'damd_multiwoz/db/train_db.json'


json_list = [attraction_json, bus_json, hospital_json, hotel_json, police_json, restaurant_json, taxi_json, train_json]


In [7]:
for file in json_list:
    with open(file, 'r') as json_file:
        data = json.load(json_file)
        print( f'{os.path.basename(file)} : {len(data)}')

attraction_db.json : 79
bus_db.json : 2828
hospital_db.json : 66
hotel_db.json : 33
police_db.json : 1
restaurant_db.json : 110
taxi_db.json : 1
train_db.json : 2828


In [18]:
query_dict = {'parking': 'yes',
            'pricerange': 'cheap',
            'type': 'hotel'}
''' 
               'hotel': {'book': {'booked': [{'name': 'the cambridge belfry',
                                              'reference': '7gawk763'}],
                                  'day': 'tuesday',
                                  'people': '6',
                                  'stay': '2'},
                         'semi': {'area': 'not mentioned',
                                  'internet': 'not mentioned',
                                  'name': 'not mentioned',
                                  'parking': 'yes',
                                  'pricerange': 'cheap',
                                  'stars': 'not mentioned',
                                  'type': 'hotel'}},

'''
def json_query(file_path, q_dict, debug=False):
    '''
    file_path: domain's db path
    q_dict: query dict
    '''
    to_match = len(q_dict)
    matched_item = []
    with open(file_path, 'r') as json_file:
        items = json.load(json_file)
        searched = 0
        match_success = 0
        for item in items:
            searched += 1
            matched = 0
            for k, v in q_dict.items():
                if item.get(k, False) == v:
                    matched += 1
            if matched == to_match:
                match_success += 1
                matched_item.append(item)
        if debug:
            print(f'available query result {match_success} / {searched}')
            pprint.pprint(matched_item)
    return matched_item

query_result = json_query(hotel_json, query_dict, debug= True)

available query result 1 / 33
[{'address': 'back lane, cambourne',
  'area': 'west',
  'id': '28',
  'internet': 'yes',
  'location': [52.2213805555556, -0.0680333333333333],
  'name': 'the cambridge belfry',
  'parking': 'yes',
  'phone': '01954714600',
  'postcode': 'cb236bw',
  'price': {'double': '60', 'single': '60'},
  'pricerange': 'cheap',
  'stars': '4',
  'takesbookings': 'yes',
  'type': 'hotel'}]


In [15]:
pprint.pprint(query_result)

{'address': '152 chesterton road',
 'area': 'north',
 'id': '32',
 'internet': 'yes',
 'location': [52.2309912, 0.1295545],
 'name': 'worth house',
 'parking': 'yes',
 'phone': '01223316074',
 'postcode': 'cb41da',
 'price': {'double': '60', 'family': '85', 'single': '49'},
 'pricerange': 'cheap',
 'stars': '4',
 'takesbookings': 'yes',
 'type': 'guesthouse'}


In [20]:
query_dict = {'area': 'centre'}
query_result_rest = json_query(restaurant_json, query_dict, debug=True)
pprint.pprint(query_result_rest)


available query result 69 / 110
[{'address': 'Regent Street City Centre',
  'area': 'centre',
  'food': 'italian',
  'id': '19210',
  'introduction': 'Pizza hut is a large chain with restaurants nationwide '
                  'offering convenience pizzas pasta and salads to eat in or '
                  'take away',
  'location': [52.20103, 0.126023],
  'name': 'pizza hut city centre',
  'phone': '01223323737',
  'postcode': 'cb21ab',
  'pricerange': 'cheap',
  'type': 'restaurant'},
 {'address': '106 Regent Street City Centre',
  'area': 'centre',
  'food': 'indian',
  'id': '19214',
  'introduction': 'curry garden serves traditional indian and bangladeshi '
                  'cuisine cooked with fresh produce delivered every day',
  'location': [52.200187, 0.126407],
  'name': 'curry garden',
  'phone': '01223302330',
  'postcode': 'cb21dp',
  'pricerange': 'expensive',
  'type': 'restaurant'},
 {'address': 'Market Hill City Centre',
  'area': 'centre',
  'food': 'italian',
  'id': '

In [4]:
# 测试 sentence transformer
pre_db_paths = {
            'attraction': 'db/attraction_db_processed.json',
            'hospital': 'db/hospital_db_processed.json',
            'hotel': 'db/hotel_db_processed.json',
            'police': 'db/police_db_processed.json',
            'restaurant': 'db/restaurant_db_processed.json',
            'taxi': 'db/taxi_db_processed.json',
            'train': 'db/train_db_processed.json',
        }

In [7]:
#data_prefix = './damd_multiwoz'
pre_dom_keys = {}
for domain, path in pre_db_paths.items():
    pre_dom_keys[domain] = {}
    #dom_path = os.path.join(data_prefix, path)
    dom_path = path
    with open (dom_path, 'r') as f:
        dom_data = json.loads(f.read().lower())
        for idx, entity in enumerate(dom_data):
            for attrb in entity:
                if attrb not in pre_dom_keys[domain].keys():
                    pre_dom_keys[domain][attrb] = 1
                else:
                    pre_dom_keys[domain][attrb] += 1
print(pre_dom_keys)
pprint.pprint(pre_dom_keys)
for domain, slots in pre_dom_keys.items():
    print(domain)
    print(slots.keys())


{'attraction': {'location': 79, 'address': 79, 'area': 79, 'price': 79, 'id': 79, 'name': 79, 'openhours': 79, 'phone': 79, 'postcode': 79, 'pricerange': 79, 'type': 79}, 'hospital': {'id': 66, 'department': 66, 'phone': 66}, 'hotel': {'location': 33, 'price': 33, 'address': 33, 'area': 33, 'internet': 33, 'parking': 33, 'id': 33, 'name': 33, 'phone': 33, 'postcode': 33, 'pricerange': 33, 'stars': 33, 'takesbookings': 33, 'type': 33}, 'police': {'id': 1, 'name': 1, 'address': 1, 'phone': 1}, 'restaurant': {'location': 110, 'address': 110, 'area': 110, 'food': 110, 'id': 110, 'introduction': 96, 'name': 110, 'phone': 107, 'postcode': 110, 'pricerange': 110, 'type': 110, 'signature': 16}, 'taxi': {'taxi_colors': 1, 'taxi_types': 1, 'taxi_phone': 1}, 'train': {'arrive': 2828, 'day': 2828, 'departure': 2828, 'destination': 2828, 'time': 2828, 'leave': 2828, 'price': 2828, 'id': 2828}}
{'attraction': {'address': 79,
                'area': 79,
                'id': 79,
                'loca

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('../all-MiniLM-L6-v2/')

#Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']


#Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

In [7]:
sentences = ['Python is an interpreted high-level general-purpose programming language.',
    'Python is dynamically-typed and garbage-collected.',
    'The quick brown fox jumps over the lazy dog.']

# Get embeddings of sentences
embeddings = model.encode(sentences)

# Compute similarities
sim = util.cos_sim(embeddings[0], embeddings[1])
print("{0:.4f}".format(sim.tolist()[0][0])) # 0.6445
sim = util.cos_sim(embeddings[0], embeddings[2])
print("{0:.4f}".format(sim.tolist()[0][0])) # 0.0365

0.6830
0.1030


In [12]:
with open (pre_db_paths['hotel'], 'r') as f:
    dom_data = json.loads(f.read().lower())
    rand_sample = random.sample(dom_data, 1)
    rand_attrbs = random.sample(rand_sample[0].keys(),3)
    print('rand_sample: ')
    pprint.pprint(rand_sample)
    print('rand_attrbs:')
    pprint.pprint(rand_attrbs)

rand_sample: 
[{'address': '63 milton road',
  'area': 'north',
  'id': '4',
  'internet': 'no',
  'location': [52.2173388888889, 0.127638888888889],
  'name': 'alpha - milton guest house',
  'parking': 'no',
  'phone': '01223311625',
  'postcode': 'cb41xa',
  'price': {'double': '80', 'single': '45'},
  'pricerange': 'moderate',
  'stars': '3',
  'takesbookings': 'yes',
  'type': 'guest house'}]
rand_attrbs:
['pricerange', 'area', 'takesbookings']


In [8]:
def convert2str(value):
    '''
    convert non str object to str
    '''
    if isinstance(value, list):
        return ', '.join(map(str, value))
    elif isinstance(value, dict):
        return ', '.join(f'{k}, {v}' for k, v in value.items())
    elif isinstance(value, str):
        return value
    else:
        print(f'value {value} type {type(value)} not considered!')

def extract_dom_data(data_path):
    with open (data_path, 'r') as f:
        data = json.loads(f.read().lower())
    slot_dict = {}
    for item in data:
        for slot, value in item.items():
            if not slot_dict.get(slot):
                slot_dict[slot] = [convert2str(value)]
            else:
                slot_dict[slot].append(convert2str(value))
    return slot_dict

hotel_dict = extract_dom_data(pre_db_paths['hotel'])
with open (pre_db_paths['hotel'], 'r') as f:
    dom_data = json.loads(f.read().lower())
    rand_item = random.choice(dom_data)
    item_num = len(dom_data)
    #rand_slots = random.sample(rand_item[0].keys(),3)
    random_slots = ['price', 'type', 'name', 'postcode', 'phone']
    print('rand_item: ')
    pprint.pprint(rand_item)
    print('\n rand_slots:')
    pprint.pprint(random_slots)
    total_scores = torch.zeros(item_num).to(device)
    for slot in random_slots:
        print()
        query = convert2str(rand_item[slot])
        corpus = hotel_dict[slot]
        corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
        query_embedding = model.encode(query, convert_to_tensor=True)
        #print(f"size of corpus: {len(corpus)}, shape of corpus_embedding: {corpus_embeddings.shape}, shape of query_embedding: {query_embedding.shape}")
        top_k = min(5, len(corpus))
        # We use cosine-similarity and torch.topk to find the highest 5 scores
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        #print(f'cos_scors shape: {cos_scores.shape}')
        total_scores += cos_scores
        #print(f'cos scores: {cos_scores}')
        top_results = torch.topk(cos_scores, k=top_k)
        #print(f'top_results: {top_results}')

        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:")

        for score, idx in zip(top_results[0], top_results[1]):
            print("index: {}".format(idx), corpus[idx], "(Score: {:.4f})".format(score))

    total_scores /= len(random_slots)
    all_top_results = torch.topk(total_scores, k=top_k)
    #print(f'top_results: {top_results}')

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    print('original item:')
    pprint.pprint(rand_item)
    for score, idx in zip(all_top_results[0], all_top_results[1]):
        print("index: {}".format(idx), "(Score: {:.4f})".format(score))
        pprint.pprint(dom_data[idx])
    print('type of idx ', all_top_results[1])
    top_index = all_top_results[1].tolist()
    top_index.pop(0)
    print('random_index ', len(top_index), random.choice(top_index))
    

rand_item: 
{'address': 'warkworth terrace',
 'area': 'east',
 'id': '31',
 'internet': 'yes',
 'location': [52.1963733, 0.1987426],
 'name': 'warkworth house',
 'parking': 'yes',
 'phone': '01223363682',
 'postcode': 'cb11ee',
 'price': {'double': '75', 'family': '95', 'single': '55'},
 'pricerange': 'moderate',
 'stars': '4',
 'takesbookings': 'yes',
 'type': 'guest house'}

 rand_slots:
['price', 'type', 'name', 'postcode', 'phone']





Query: double, 75, family, 95, single, 55

Top 5 most similar sentences in corpus:
index: 31 double, 75, family, 95, single, 55 (Score: 1.0000)
index: 10 double, 75, family, 95, single, 65 (Score: 0.9926)
index: 21 double, 75, family, 105, single, 50 (Score: 0.9606)
index: 1 double, 75, family, 105, single, 50 (Score: 0.9606)
index: 11 double, 75, family, 90, single, 50 (Score: 0.9533)





Query: guest house

Top 5 most similar sentences in corpus:
index: 1 guest house (Score: 1.0000)
index: 0 guest house (Score: 1.0000)
index: 2 guest house (Score

In [4]:
# check restaurant domain
'''
'restaurant': {'address': 110,
                'area': 110,
                'food': 110,
                'id': 110,
                'introduction': 96,
                'location': 110,
                'name': 110,
                'phone': 107,
                'postcode': 110,
                'pricerange': 110,
                'signature': 16,
                'type': 110},
'''
with open (pre_db_paths['restaurant'], 'r') as f:
    rest_data = json.loads(f.read().lower())
    random_item = random.choice(rest_data)
    print('*********** A Random Item **************')
    pprint.pprint(random_item)
    debug_num = 4
    print('*********** Debug Item ***********')
    for item in rest_data:
        if debug_num < 0:
            break
        if 'phone' not in item.keys():
            pprint.pprint(item)
            debug_num -= 1

*********** A Random Item **************
{'address': '47 - 53 regent street',
 'area': 'centre',
 'food': 'italian',
 'id': '29652',
 'introduction': '',
 'location': [52.201116, 0.125712],
 'name': 'zizzi cambridge',
 'phone': '01223365599',
 'postcode': 'cb21ab',
 'pricerange': 'cheap',
 'signature': 'piccante rustica pizza , a spicy sausage salami mascarpone and '
              'roquito chilli',
 'type': 'restaurant'}
*********** Debug Item ***********
{'address': '12 st . johns street city centre',
 'area': 'centre',
 'food': 'chinese',
 'id': '19228',
 'introduction': 'ugly duckling serves a variety of chinese dishes to eat in '
                 'or take away they also offer 10 percent discount on takeaway '
                 'orders',
 'location': [52.208055, 0.118397],
 'name': 'ugly duckling',
 'postcode': 'cb21tw',
 'pricerange': 'expensive',
 'type': 'restaurant'}
{'address': '196 mill road city centre',
 'area': 'centre',
 'food': 'turkish',
 'id': '19189',
 'introduction': '

In [17]:
import os, sys
print(os.getcwd())

/home/fanghongming/CASPI


In [31]:
path_to_add = './damd_multiwoz/'
sys.path.append(path_to_add)
sys.path.insert(0, path_to_add)

from damd_multiwoz.preprocess import extract_dom_data
rest_data = extract_dom_data(pre_db_paths['restaurant'])
print(type(rest_data))
print(rest_data.keys())

for slot, value in rest_data.items():
    print(slot, len(value))

value_set_path = './damd_multiwoz/db/value_set_processed.json'
with open (value_set_path, 'r') as f:
    data = json.loads(f.read().lower())
    print(type(data['hotel']['stay']))

<class 'dict'>
dict_keys(['location', 'address', 'area', 'food', 'id', 'introduction', 'name', 'phone', 'postcode', 'pricerange', 'type', 'signature'])
location 110
address 110
area 110
food 110
id 110
introduction 110
name 110
phone 110
postcode 110
pricerange 110
type 110
signature 110
<class 'list'>


In [8]:
from collections import OrderedDict
d = OrderedDict()
d['phone'] = 1234
d['mobile'] = 12345
print(d)
d['mobile'] = 123
print(d)

OrderedDict([('phone', 1234), ('mobile', 12345)])
OrderedDict([('phone', 1234), ('mobile', 123)])


In [5]:

e = OrderedDict([('pricerange', 'cheap'), ('type', 'hotel'), ('parking', 'yes')])

In [6]:
d['hotel'] = e

In [15]:
import os, json
with open ('damd_multiwoz/db/value_set_processed.json', 'r') as f:
    data = json.load(f)

    for dom, slot_value in data.items():
        print('*********** domain ************')
        print('//',dom, '//')
        for slot, value in slot_value.items():
            print(slot)


*********** domain ************
// police //
*********** domain ************
// taxi //
leave
destination
departure
arrive
day
*********** domain ************
// restaurant //
name
area
pricerange
food
time
day
people
*********** domain ************
// attraction //
name
area
type
*********** domain ************
// hotel //
name
internet
area
parking
stars
type
pricerange
stay
day
people
*********** domain ************
// hospital //
department
*********** domain ************
// train //
departure
day
arrive
destination
leave
people


In [22]:
# debug 326 problem
# sng0732.json
# pmul2201.json
print(data['sng0732.json']['log'])
print(data['sng0732.json']['log'][0])

[{'text': 'hi can you help me find a cheap restaurant in the centre of town? ', 'metadata': {}}, {'text': 'i certainly can. we have 15 inexpensive restaurants in the center of town. was there a particular type of cuisine you wanted?', 'metadata': {'taxi': {'book': {'booked': []}, 'semi': {'leaveat': '', 'destination': '', 'departure': '', 'arriveby': ''}}, 'police': {'book': {'booked': []}, 'semi': {}}, 'restaurant': {'book': {'booked': [], 'time': '', 'day': '', 'people': ''}, 'semi': {'food': 'not mentioned', 'pricerange': 'cheap', 'name': 'not mentioned', 'area': 'centre'}}, 'hospital': {'book': {'booked': []}, 'semi': {'department': ''}}, 'hotel': {'book': {'booked': [], 'stay': '', 'day': '', 'people': ''}, 'semi': {'name': '', 'area': '', 'parking': '', 'pricerange': '', 'stars': '', 'internet': '', 'type': ''}}, 'attraction': {'book': {'booked': []}, 'semi': {'type': '', 'name': '', 'area': ''}}, 'train': {'book': {'booked': [], 'people': ''}, 'semi': {'leaveat': '', 'destinatio

In [28]:
def show_log(data, dial_fn):
    turn_num = 0
    for turn in data[dial_fn]['log']:
        print('Turn {}'.format(turn_num))
        print('***text***')
        print(turn['text'])
        print('***metadata***')
        pprint.pprint(turn['metadata'])
        turn_num += 1
show_log(data, 'sng0732.json')

Turn 0
***text***
hi can you help me find a cheap restaurant in the centre of town? 
***metadata***
{}
Turn 1
***text***
i certainly can. we have 15 inexpensive restaurants in the center of town. was there a particular type of cuisine you wanted?
***metadata***
{'attraction': {'book': {'booked': []},
                'semi': {'area': '', 'name': '', 'type': ''}},
 'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
 'hotel': {'book': {'booked': [], 'day': '', 'people': '', 'stay': ''},
           'semi': {'area': '',
                    'internet': '',
                    'name': '',
                    'parking': '',
                    'pricerange': '',
                    'stars': '',
                    'type': ''}},
 'police': {'book': {'booked': []}, 'semi': {}},
 'restaurant': {'book': {'booked': [], 'day': '', 'people': '', 'time': ''},
                'semi': {'area': 'centre',
                         'food': 'not mentioned',
                         'name': 'not

In [29]:
show_log(data, 'pmul2201.json')

Turn 0
***text***
i'm looking for a museum in east area. 
***metadata***
{}
Turn 1
***text***
there are four museums. three are free and one is 5 pounds. 
***metadata***
{'attraction': {'book': {'booked': []},
                'semi': {'area': 'east',
                         'name': 'not mentioned',
                         'type': 'museum'}},
 'bus': {'book': {'booked': [], 'people': ''},
         'semi': {'arriveby': '',
                  'day': '',
                  'departure': '',
                  'destination': '',
                  'leaveat': ''}},
 'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
 'hotel': {'book': {'booked': [], 'day': '', 'people': '', 'stay': ''},
           'semi': {'area': '',
                    'internet': '',
                    'name': '',
                    'parking': '',
                    'pricerange': '',
                    'stars': '',
                    'type': ''}},
 'police': {'book': {'booked': []}, 'semi': {}},
 'restaur

In [2]:
a = 'My long-term goal is to specialize in applying AI in industry, particularly in handling extensive data, model training, and ensuring result safety. Despite my strong foundation in AI algorithms, I have identified a need for more practical experience and methodologies for real-world applications.After researching programs extensively, I believe DTU\'s Computer Science and Engineering program, with a focus on Artificial Intelligence and Algorithms, is perfect for me. It offers both theoretical and practical support for implementing algorithms in real-world scenarios, including large datasets and distributed systems. What sets it apart is the inclusion of logic and formalization courses, which are rare elsewhere. Additionally, I\'m excited to explore courses in cybersecurity, model check  to broaden my knowledge in these areas.DTU also has outstanding reputation and strong ties to the industry, which present many opportunities for me to apply the knowledge and skills I will gain during my studies to real-world AI challenges, which aligns perfectly with my goal to advance my expertise and contribute to the AI industry.'
print(len(a))

1130


In [1]:
from transformers import PretrainedConfig
default_config = PretrainedConfig()
print(default_config.num_labels)

2


In [2]:
# 计算平均因果效应
# log_2023-10-31_sd68693
match_bspn = [84.88, 86.99, 83.18, 82.68, 87.19, 88.39, 86.49, 85.39, 85.59, 86.49]
success_bspn = [75.58, 74.97, 72.27, 72.37, 77.28, 77.98, 74.67, 75.18, 73.97, 76.28]
bleu_bspn = [19.01, 19.18, 18.91, 19.77, 19.65, 19.22, 19.14, 17.88, 19.08, 18.86]
match_no_bspn = [76.26, 72.77, 78.28, 74.47, 79.68]
success_no_bspn = [59.86, 59.56, 60.06, 58.06, 63.86]
print('match_bspn avg:',sum(match_bspn)/len(match_bspn))
print('success_bspn avg:',sum(success_bspn)/len(success_bspn))
print('bleu_bspn avg:',sum(bleu_bspn)/len(bleu_bspn))
print('match_no_bspn avg:',sum(match_no_bspn)/len(match_no_bspn))
print('success_no_bspn avg:',sum(success_no_bspn)/len(success_no_bspn))

match_bspn avg: 85.727
success_bspn avg: 75.05499999999999
bleu_bspn avg: 19.07
match_no_bspn avg: 76.292
success_no_bspn avg: 60.28000000000001


In [4]:
# 计算 cntfact 0.05 的对比学习效果 (hz)
# 截止 seed 94
match = [84.78, 92.39, 83.88, 91.29, 84.08]
success = [73.77, 79.98, 70.97, 77.18, 70.67]
bleu = [19.27, 20.18, 19.59, 17.97, 18.86]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 87.284
success avg: 74.514
bleu avg: 19.174


In [2]:
# 计算cntfact 0.1 的对比学习效果 
match = [90.39, 88.39, 85.29, 86.79, 86.79, 83.88, 85.49, 93.19] 
success = [75.38, 79.88, 76.68, 73.77, 77.28, 74.97, 70.17, 78.68]
bleu = [18.02, 19.54, 19.35, 19.41, 18.41, 18.72, 18.11, 16.94]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 87.52625
success avg: 75.85125
bleu avg: 18.5625


In [1]:
# 计算 cntfact 0.2 的对比学习效果 (hz)
match = [85.49, 84.98, 87.39, 90.39, 88.09, 85.79, 84.48, 92.79, 85.69, 84.68]
success = [74.47, 75.98, 73.77, 77.48, 77.18, 76.88, 74.97, 75.58, 74.87, 75.48]
bleu = [19.36, 19.56, 19.48, 18.03, 17.55, 18.98, 19.07, 17.35, 19.17, 19.60]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 86.977
success avg: 75.66600000000001
bleu avg: 18.815


In [1]:
# 计算cntfact 0.4 的对比学习效果 (hz)
# 90 95  killed
# 79.18  success: 67.67  bleu: 17.56
match = [91.29, 87.59, 85.69, 89.19, 87.89, 86.99, 89.99, 87.99, 82.68, 79.18]
success = [80.08, 74.97, 72.67, 81.18, 77.68, 77.48, 79.78, 76.78, 72.27, 67.67]
bleu = [19.71, 18.59, 19.12, 18.20, 17.30, 18.24, 19.16, 18.81, 19.15, 17.56]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 86.848
success avg: 76.056
bleu avg: 18.584


In [3]:
# 计算 cntfact 0.6 的对比学习效果 
# 94, 95killed -> updated
match = [84.88, 84.28, 91.19, 82.38, 92.89, 91.29, 89.09, 85.39, 83.08, 87.79]
success = [74.87, 71.17, 80.58, 73.17, 77.68, 79.88, 80.28, 74.37, 75.48, 78.28]
bleu = [19.36, 19.87, 19.44, 19.26, 16.55, 16.86, 18.73, 18.95, 19.75, 20.41]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 87.226
success avg: 76.576
bleu avg: 18.918


In [3]:
# 计算 cntfact 0.8 的对比学习效果 
match = [85.89, 85.49, 85.89, 84.98, 90.39, 85,79, 90.69, 80.78, 92.09, 88.29]
success = [74.67, 77.38, 75.28, 72.77, 80.98, 76.18, 72.27, 81.34, 81.48, 75.98]
bleu = [19.00, 19.32, 19.10, 19.13, 17.43, 19.41, 15.77, 19.36, 19.08, 20.03]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 86.22636363636363
success avg: 76.83300000000001
bleu avg: 18.762999999999998


In [3]:
# 计算 cntfact K=1 的对比学习效果 
# 91, 93, 94 killed
match = [78.98, 86.59, 88.39, 90.79, 91.69, 90.59, 89.69, 82.58, 85.29, 86.79, 88.69]
success = [71.17, 76.98, 78.58, 69.87, 79.78, 81.08, 77.08, 72.97, 74.37, 74.67, 77.38]
bleu = [19.96, 19.46, 18.99, 15.79, 17.28, 18.77, 18.76, 19.42, 18.44, 18.98, 18.06]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 87.2790909090909
success avg: 75.81181818181818
bleu avg: 18.537272727272725


In [3]:
# 计算 cntfact K=2 的对比学习效果 
# epoch/early_stop = 20/3
# 72.67  success: 55.76  bleu: 18.33
match = [72.27, 88.09, 92.59, 87.19, 78.88, 78.38, 90.09, 91.19, 82.48, 72.67]
success = [57.46, 72.37, 66.57, 75.68, 67.77, 64.46, 70.87, 78.58, 56.56, 55.76]
bleu = [18.93, 15.47, 16.79, 18.42, 18.30, 19.37, 18.18, 18.18, 15.15, 18.33]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

# epoch/early_stop = 100/5
#    : 88.39  success: 72.57  bleu: 19.52
match = [84.38, 84.88, 84.18, 84.88, 81.68, 88.49, 85.99, 80.18, 82.58, 88.39]
success = [74.17, 72.77, 75.08, 73.37, 66.17, 78.88, 75.28, 69.07, 72.57, 72.57]
bleu = [18.50, 18.41, 19.26, 18.52, 18.18, 19.44, 19.72, 18.70, 19.75, 19.52]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 83.38300000000001
success avg: 66.60799999999999
bleu avg: 17.712
match avg: 84.56300000000002
success avg: 72.99299999999998
bleu avg: 19.0


In [1]:
# 计算 cntfact K=3 的对比学习效果 
#    85.79  success: 76.98  bleu: 19.67

# seed 91: log_2024-01-11-23-01-28_train_all_all_no_aug_cntfact_3_bspn_sd68691
# 84.18  success: 73.77  bleu: 19.37
match = [84.18, 84.88, 92.79, 88.09, 89.89, 83.38, 85.79, 82.98, 88.09, 86.89]
success = [73.77, 72.77, 79.18, 77.28, 77.18, 74.87, 76.98, 61.46, 64.86, 76.38]
bleu = [19.67, 19.60, 17.44, 20.15, 19.48, 20.46, 19.67, 18.26, 15.70, 19.84]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 86.696
success avg: 73.473
bleu avg: 19.027


In [2]:
# 计算cntfact K=4 的对比学习效果
# epoch/early_stop = 20/3
#   84.88  success: 74.67  bleu: 20.03
# :  81.98  success: 70.17  bleu: 19.79

match = [88.89, 84.88, 83.58, 87.39, 85.19, 86.19, 86.39, 83.08, 85.49, 81.98]
success = [76.08, 74.67, 73.47, 75.58, 70.37, 71.17, 77.18, 69.97, 74.27, 70.07]
bleu = [18.31, 20.03, 19.09, 19.47, 19.03, 16.33, 19.37, 17.96, 18.45, 19.79]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

# epoch/early_stop = 100/5
#  81.68  success: 67.47  bleu: 17.40
match = [85.49, 84.78, 79.98, 86.09, 81.98, 93.09, 88.09, 85.39, 77.88, 81.68]
success = [72.27, 73.37, 64.86, 73.37, 65.77, 79.58, 76.88, 75.98, 65.97, 67.47]
bleu = [18.61, 18.72, 17.50, 18.68, 17.39, 19.11, 19.17, 18.88, 19.35, 17.40] 
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))


match avg: 85.306
success avg: 73.28299999999999
bleu avg: 18.782999999999998
match avg: 84.44500000000001
success avg: 71.55199999999999
bleu avg: 18.481


In [1]:
# 计算 cntfact K=5 的对比学习效果 
match = [88.19, 91.19, 86.39, 87.59, 83.98, 85.89, 84.88, 88.69, 86.19, 90.79]
success = [76.88, 81.98, 76.18, 70.47, 76.38, 77.48, 77.18, 77.88, 76.48, 79.08]
bleu = [20.10, 19.32, 19.97, 19.74, 19.53, 20.07, 19.51, 20.15, 19.38, 19.19]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 87.378
success avg: 76.999
bleu avg: 19.695999999999998


In [1]:
# 计算 REINFORCE 算法 只用 success, match 做reward 的效果
match = [96.00, 95.20, 97.30, 97.20, 94.09, 96.50, 96.40, 93.79, 95.90, 97.00]
success = [81.58, 80.78, 82.68, 83.78, 74.47, 79.78, 82.48, 77.18, 82.38, 86.29]
bleu = [16.60, 17.80, 17.12, 17.38, 17.23, 16.37, 15.84, 17.44, 16.00, 17.23]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 95.93799999999999
success avg: 81.14
bleu avg: 16.901000000000003


In [1]:
# 计算 REINFORCE 算法 用 success, match cntfact_sample=-0.1 做reward 的效果  
match = [96.60, 96.80, 96.60, 95.60, 96.70, 96.60, 96.80, 96.90, 96.50, 96.60]
success = [84.38, 80.68, 83.48, 80.68, 76.48, 82.28, 80.88, 84.18, 85.29, 84.58]
bleu = [16.57, 17.86, 16.20, 16.38, 15.02, 17.40, 17.35, 17.54, 18.23, 17.30]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 96.57
success avg: 82.291
bleu avg: 16.984999999999996


In [1]:
# 计算 REINFORCE 算法 用 success, match cntfact_sample = -0.5, actin sim_score 做reward 的效果
#match: match: 94.19  success: 82.08  bleu: 15.06
match = [96.60, 95.10, 95.40, 96.30, 97.30, 92.09, 96.50, 96.70, 96.60, 94.19]
success = [82.18, 76.98, 82.08, 81.28, 79.68, 82.08, 84.98, 82.18, 83.58, 82.08]
bleu = [17.77, 18.72, 17.26, 17.65, 15.93, 18.39, 16.82, 19.37, 16.18, 15.06]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 95.67800000000003
success avg: 81.71000000000001
bleu avg: 17.315000000000005


In [1]:
# BCQ 算法 隐动作输出 加到 REINFORCE 的效果

#   95.90  success: 77.88  bleu: 16.96
match = [96.50, 97.20, 93.89, 96.80, 95.90]
success = [78.78, 87.89, 75.88, 82.28, 77.88]
bleu = [17.27, 15.89, 17.39, 17.94, 16.96]

print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 96.05799999999999
success avg: 80.542
bleu avg: 17.089999999999996


In [1]:
# 计算cntfact 0.1 的平均因果效应 (bsdx，没效果)
match = [86.89, 90.99, 92.99, 84.58, 83.98, 89.39, 92.19, 84.88, 86.79, 85.39]
success = [74.67, 75.88, 78.88, 73.37, 70.97, 78.78, 76.78, 73.47, 72.27, 70.07]
bleu = [18.70, 18.61, 15.56, 19.32, 19.15, 17.90, 18.96, 19.17, 19.92, 19.59]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 87.80699999999999
success avg: 74.51399999999998
bleu avg: 18.688000000000002


In [1]:
# 计算cntfact 1 的平均因果效应 (bsdx, 没效果)
match = [82.38, 89.99, 84.18, 85.79, 88.19, 85.89, 91.29, 85.29, 86.59, 86.69]
success = [72.37, 75.68, 74.47, 72.67, 79.28, 73.77, 73.67, 76.68, 76.78, 75.48]
bleu = [18.66, 19.44, 18.30, 18.55, 17.38, 19.23, 15.45, 19.05, 19.05, 19.24]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 86.628
success avg: 75.085
bleu avg: 18.435000000000002


In [4]:
from info_nce import InfoNCE, info_nce
import torch
loss = InfoNCE(negative_mode='unpaired') # negative_mode='unpaired' is the default value
batch_size, num_negative, embedding_size = 32, 48, 128
query = torch.randn(batch_size, embedding_size)
positive_key = torch.randn(batch_size, embedding_size)
negative_keys = torch.randn(num_negative, embedding_size)
output = loss(query, positive_key, negative_keys)
print(output.item())

4.541757583618164


In [1]:
# 计算cntfact 0.1 的效果，只用cntfact_bspn (bspn，有效果) 
match = [70.27, 74.07, 77.28, 73.17, 78.68, 79.58, 70.97, 72.97, 76.48, 73.17]
success = [60.66, 56.86, 65,27, 60.96, 66.67, 65.97, 62.56, 62.46, 65.77, 61.36]
bleu = [19.13, 17.63, 17.16, 18.41, 17.45, 18.33, 19.06, 17.96, 18.66, 18.91]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 74.664
success avg: 59.57
bleu avg: 18.27


In [1]:
# 计算cntfact 0.4 的效果，只用cntfact_bspn (bspn，有效果) 
match = [71.67, 71.17, 74.07, 70.27, 76.38, 68.37, 69.67, 71.37]
success = [60,16, 59.26, 58.56, 59.76, 64.66, 57.96, 59.16, 57.36]
bleu = [18.75, 19.42, 17.21, 18.67, 17.69, 18.68, 17.82, 16.56]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 71.62125
success avg: 54.74666666666667
bleu avg: 18.1


In [1]:
# 计算cntfact 1 的效果，只用cntfact_bspn (bspn，有效果) 
match = [53.25, 52.45, 52.45, 54.25, 50.75, 50.25, 51.55, 48.55, 51.45, 56.56]
success = [40.24, 42.54, 42.64, 45.95, 40.84, 42.34, 42.34, 41.94, 41.94, 47.15]
bleu = [18.05, 17.22, 18.30, 15.68, 16.26, 17.42, 17.20, 17.90, 17.09, 18.62]
print('match avg:',sum(match)/len(match))
print('success avg:',sum(success)/len(success))
print('bleu avg:',sum(bleu)/len(bleu))

match avg: 52.150999999999996
success avg: 42.791999999999994
bleu avg: 17.374000000000002


In [ ]:
# 加reward 效果
# match: 97.00  success: 84.48  bleu: 17.21

In [4]:
# debug Multi cntfact same bspn problem
import os
debug_file = os.path.join(save_path_exp, 'cntfact_data_for_damd_K_5_debug.json')
print(debug_file)
with open(debug_file, 'r', encoding='utf-8') as file:
    debug_data = json.load(file)
print(len(debug_data))
for turn in debug_data['sng01856']['log']:
    print(turn['cntfact_constraint_max'])


damd_multiwoz/data/multi-woz-processed/cntfact_data_for_damd_K_5_debug.json
10433
['[hotel] pricerange expensive type hotel', '[hotel] pricerange cheap type guest house', '[hotel] pricerange expensive type hotel', '[hotel] pricerange expensive type hotel', '[hotel] pricerange expensive type hotel']
['[hotel] pricerange expensive type hotel parking yes', '[hotel] pricerange cheap type guest house parking yes', '[hotel] pricerange expensive type hotel parking yes', '[hotel] pricerange expensive type hotel parking yes', '[hotel] pricerange expensive type hotel parking yes']
['[hotel] pricerange expensive type hotel parking yes stay 2 day monday people 7', '[hotel] pricerange cheap type guest house parking yes stay 4 day wednesday people 5', '[hotel] pricerange expensive type hotel parking yes stay 1 day thursday people 8', '[hotel] pricerange expensive type hotel parking yes stay 5 day saturday people 4', '[hotel] pricerange expensive type hotel parking yes stay 6 day sunday people 3']
['

In [24]:
print(type(debug_data))
print(type(debug_data['sng01856']['log']))
pprint.pprint(debug_data['sng01856']['log'][0])
print(len(debug_data['sng01856']['log']))

<class 'dict'>
<class 'list'>
{'cntfact_cons_delex_max': ['[hotel] pricerange type',
                            '[hotel] pricerange type',
                            '[hotel] pricerange type',
                            '[hotel] pricerange type',
                            '[hotel] pricerange type'],
 'cntfact_constraint_max': ['[hotel] pricerange expensive type hotel',
                            '[hotel] pricerange expensive type hotel',
                            '[hotel] pricerange expensive type hotel',
                            '[hotel] pricerange expensive type hotel',
                            '[hotel] pricerange expensive type hotel'],
 'cons_delex': '[hotel] pricerange type',
 'constraint': '[hotel] pricerange cheap type hotel',
 'match': '5',
 'pointer': '0,0,0,1,0,0',
 'resp': 'okay , do you have a specific area you want to stay in ?',
 'resp_nodelex': 'okay , do you have a specific area you want to stay in ?',
 'sys_act': '[hotel] [request] area',
 'turn_domain': 

In [ ]:
checked_fn = 0
diff_fn = 0
for fn, dial in debug_data.items():
    checked_fn += 1
    for turn in dial['log']:
        all_same = all(element == turn['cntfact_constraint_max'][0] for element in turn['cntfact_constraint_max'])
        if not all_same:
            print(f'current fn: {fn}')
            print(f'current cntfact_constraint:' )
            print(turn['cntfact_constraint_max'])
            diff_fn += 1
        if all_same and checked_fn % 1000 == 0:
            print(turn['cntfact_constraint_max'])

print('result:',diff_fn, checked_fn)

        

In [2]:
enable_multi_cntfact= False
item = 'cntfact_bspn'
res = enable_multi_cntfact and item == 'cntfact_bspn' or item == 'cntfact_bsdx'
print(res)

False


In [5]:
reward_file_list1 = os.path.join(save_path_exp, 'data_for_damd_reward_1.json')
reward_file_list2 = os.path.join(save_path_exp, 'data_for_damd_reward_2.json')
with open (reward_file_list1, 'r') as f:
    data = json.loads(f.read().lower())
    print(len(data))
    print((list(data.keys()))[159])

with open (reward_file_list2, 'r') as f:
    data = json.loads(f.read().lower())
    print(len(data))
    print((list(data.keys()))[159])


9434
mul1541
9434
mul1541


In [3]:
from datetime import datetime

# 获取当前时间
current_time = datetime.now()

# 格式化时间
formatted_time = current_time.strftime("%Y-%m-%d")

# 输出格式化的时间
print(formatted_time)


2024-01-30


In [2]:
# 测试 reward_report 的每一份是否就是 valid 的对应输出结果
import pandas as pd
reward_path = 'damd_multiwoz/data/multi-woz-oppe/reward/debug_bspn_cntfact_reward_report_10_soft_{}_ratio_5_dp.csv'
fns = [[] for _ in range(10)]
total_fn = 0
for i in range(10):
    data_path = reward_path.format(i)
    df = pd.read_csv(data_path)
    cur_fn = fns[i]
    for _,row in df.iterrows():
        dial_id = row['dial_id']
        if dial_id not in cur_fn:
            cur_fn.append(dial_id)
    print(f"{i} val dataset's total dials: {len(cur_fn)}")
    total_fn += len(cur_fn)
print(f"total fn: {total_fn}")
    

0 val dataset's total dials: 844
1 val dataset's total dials: 844
2 val dataset's total dials: 840
3 val dataset's total dials: 843
4 val dataset's total dials: 843
5 val dataset's total dials: 842
6 val dataset's total dials: 843
7 val dataset's total dials: 842
8 val dataset's total dials: 844
9 val dataset's total dials: 843
total fn: 8428


In [2]:
import json, os
from collections import defaultdict
root_path = './damd_multiwoz/data'
test_fn_json_path = os.path.join(root_path,'multi-woz/testListFile.json')
valid_fn_json_path = os.path.join(root_path,'multi-woz/valListFile.json')

test_fn = set(open(test_fn_json_path,'r').read().lower().replace('.json','').split())
valid_fn = set(open(valid_fn_json_path,'r').read().lower().replace('.json','').split())

data_for_damd = json.loads(open(os.path.join(root_path,'multi-woz-processed/cntfact_data_for_damd_K_5_debug.json'),'r').read())
data_for_damd_only_train = {
    fn:v for fn,v in data_for_damd.items() if fn not in test_fn and fn not in valid_fn
}
print('Train filtered/unfiltered={}/{}'.format(len(data_for_damd_only_train),len(data_for_damd)))

data_for_damd_only_val = {
    fn:v for fn,v in data_for_damd.items() if fn in valid_fn
}
print('Val filtered/unfiltered={}/{}'.format(len(data_for_damd_only_val),len(data_for_damd)))


Train filtered/unfiltered=8434/10433
Val filtered/unfiltered=999/10433


In [13]:
def get_state(fn, fn_tn_state, tn):
    turn_state = fn_tn_state[fn][tn]
    return turn_state

def get_turn_state(full_state,turn_domain):
    all_domain = ['[police]', '[taxi]', '[restaurant]', '[attraction]', '[hotel]', '[hospital]', '[train]', '[general]']
    if turn_domain not in full_state:
        return turn_domain
    turn_domain_idx = full_state.index(turn_domain)
    end_idx = len(full_state)
    for domain in all_domain:
        if turn_domain!=domain:
            if domain in full_state:
                domain_idx = full_state.index(domain)
                if domain_idx>turn_domain_idx and domain_idx<end_idx:
                    end_idx=domain_idx
                    
    return full_state[turn_domain_idx:end_idx].strip()


def get_act(data_for_damd):
    act_dict = {}
    total_turn = 0
    for fn,dia in data_for_damd.items():
        if fn not in test_fn:
            prev_act = None
            for turn_num,log in enumerate(dia['log']): 
                act = log['sys_act']
                if act not in act_dict:
                    act_dict[act] = 1
                else:
                    act_dict[act] += 1
                
                if len(act_dict) % 4000 == 0:
                    print(log.keys())
                    print(act, act_dict[act])
                total_turn += 1
    return total_turn, act_dict

In [14]:

total_act, act_dict = get_act(data_for_damd_only_train)
print(len(act_dict))
print(total_act)

dict_keys(['user', 'user_delex', 'resp', 'resp_nodelex', 'pointer', 'match', 'constraint', 'cons_delex', 'cntfact_constraint_max', 'cntfact_cons_delex_max', 'sys_act', 'turn_num', 'turn_domain'])
[attraction] [inform] price area type choice [general] [reqmore] 1
dict_keys(['user', 'user_delex', 'resp', 'resp_nodelex', 'pointer', 'match', 'constraint', 'cons_delex', 'cntfact_constraint_max', 'cntfact_cons_delex_max', 'sys_act', 'turn_num', 'turn_domain'])
[attraction] [inform] name phone postcode 2
dict_keys(['user', 'user_delex', 'resp', 'resp_nodelex', 'pointer', 'match', 'constraint', 'cons_delex', 'cntfact_constraint_max', 'cntfact_cons_delex_max', 'sys_act', 'turn_num', 'turn_domain'])
[hotel] [inform] choice [request] parking 3
dict_keys(['user', 'user_delex', 'resp', 'resp_nodelex', 'pointer', 'match', 'constraint', 'cons_delex', 'cntfact_constraint_max', 'cntfact_cons_delex_max', 'sys_act', 'turn_num', 'turn_domain'])
[restaurant] [recommend] name [inform] price choice food [off

In [18]:
huge_act = 0
for k, num in act_dict.items():
    if num >= 50 :
        huge_act += 1
        print(k)
print(huge_act)


[hotel] [request] area
[hotel] [offerbooked] reference [general] [reqmore]
[general] [bye]

[general] [welcome]
[general] [welcome] [bye]
[train] [request] day
[train] [offerbooked] reference
[hotel] [recommend] name
[hotel] [request] day
[hotel] [offerbooked] reference
[general] [greet]
[train] [offerbooked] reference price
[train] [request] destination
[train] [offerbook]
[hotel] [inform] choice [request] price
[hotel] [offerbook]
[hotel] [request] stay
[hotel] [offerbooked] reference name [general] [reqmore]
[hospital] [inform] phone
[attraction] [inform] choice [request] area
[restaurant] [inform] choice [request] food
[general] [greet] [general] [bye]
[train] [inform] arrive id leave [offerbook]
[train] [request] people
[train] [request] departure
[train] [inform] price
[hotel] [inform] choice type [request] area
[hotel] [request] people
[general] [welcome] [reqmore]
[restaurant] [recommend] name [offerbook]
[hotel] [inform] choice [request] area
[general] [reqmore]
[restaurant] [

In [2]:
import torch

def scale_to_range(tensor, range_min=-1, range_max=1):
    """
    将输入张量的每个特征缩放到指定范围内。
    
    参数:
    - tensor: 输入张量，形状为 [batch_size, hidden_size]
    - range_min: 缩放范围的最小值
    - range_max: 缩放范围的最大值
    
    返回:
    - 缩放后的张量
    """
    # 按批次（dim=0）找到最小值和最大值
    min_vals = torch.min(tensor, dim=1, keepdim=True)[0]
    max_vals = torch.max(tensor, dim=1, keepdim=True)[0]
    
    # 应用缩放公式 X_std = (X - X.min) / (X.max - X.min)
    # X_scaled = X_std * (max - min) + min
    tensor_std = (tensor - min_vals) / (max_vals - min_vals)
    tensor_scaled = tensor_std * (range_max - range_min) + range_min
    
    return tensor_scaled

# 示例
batch_size = 5
hidden_size = 10
tensor = torch.randn(batch_size, hidden_size)
tensor_scaled = scale_to_range(tensor, -1, 1)

print("原始张量:\n", tensor)
print("缩放后的张量:\n", tensor_scaled)

/home/fanghongming/anaconda3/envs/CASPI/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


原始张量:
 tensor([[ 1.2720,  0.6685, -0.4115, -0.2020, -0.3793, -1.4039,  0.1285,  1.2606,
          0.6608,  0.4735],
        [-2.0807,  1.1757, -0.8865, -0.5106,  0.1596, -0.0890,  0.5765,  0.0403,
          0.2696,  0.0545],
        [-2.0280,  1.7506, -2.8344,  0.2035,  0.0203,  1.9086, -1.3921,  0.2872,
         -0.4634,  0.4250],
        [-0.6950,  1.5287, -0.3946,  1.0442,  0.0699,  1.4200,  0.8314, -0.4021,
         -1.0692,  0.5420],
        [ 1.9168, -0.3676, -0.2176,  0.8406,  1.6432, -0.1459,  0.1311,  0.6658,
          1.5627,  1.4501]])
缩放后的张量:
 tensor([[ 1.0000e+00,  5.4893e-01, -2.5829e-01, -1.0167e-01, -2.3418e-01,
         -1.0000e+00,  1.4532e-01,  9.9153e-01,  5.4316e-01,  4.0320e-01],
        [-1.0000e+00,  1.0000e+00, -2.6655e-01, -3.5724e-02,  3.7594e-01,
          2.2325e-01,  6.3197e-01,  3.0265e-01,  4.4349e-01,  3.1140e-01],
        [-6.5994e-01,  9.3339e-01, -1.0000e+00,  2.8100e-01,  2.0374e-01,
          1.0000e+00, -3.9179e-01,  3.1631e-01, -2.1231e-04,  3.74

In [20]:
import sys
import importlib

# 对mymodule进行了修改之后，重新加载它
import utils
importlib.reload(utils)
from utils import ReplayBuffer
# 添加damd_multiwoz目录到模块搜索路径
sys.path.insert(0, './damd_multiwoz')

# 现在你可以直接导入utils模块中的ReplayBuffer


path = './experiments/20240319_all_debug_generate_bcq_train_K_5_sd20240316_lr0.005_bs128_sp3_dc3_actTrue_0.05_hashbgBEdUlfPVqA/replay.json'
buffer = ReplayBuffer()
buffer.load(path)
buffer.show('state',200001,200020)

Replay Buffer loaded with 755924 elements.
Data loaded. Train size: 680331, Validation size: 75593
********** state *********
[hotel] stars 4
[train] day thursday departure leicester
[attraction] type college
[hotel] name alpha - milton guest house
[restaurant] pricerange moderate area south
[taxi] departure clowns cafe
[train] leave 12:30 departure peterborough
[train] destination stevenage departure cambridge
[hotel] pricerange expensive stars 4
len of buffer now: 755924


In [21]:
buffer.show('action',200001,200020)

********** action *********
[hotel] [inform] area
[train] [inform] choice [request] [request] [request] leave
[attraction] [inform] name choice area
[hotel] [inform] area name type
[restaurant] [inform] name name name name food
[taxi] [request] leave
[train] [request] choice [request] destination leave
[train] [request] choice [request] leave
[hotel] [inform] area
len of buffer now: 755924


In [37]:
import numpy as np
buffer.show('reward',200001,2000020)
print(buffer.reward[200000:200020])
print(np.sum(buffer.reward)/buffer.reward.shape[0])

********** reward *********
[[0.        ]
 [0.        ]
 [0.        ]
 ...
 [3.5       ]
 [3.2632578 ]
 [3.28600365]]
len of buffer now: 755924
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [3.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]]
0.940192963739386


In [35]:
buffer.show('not_done',200001,200020)
print(buffer.not_done[200000:200020])
print(np.sum(buffer.not_done))

********** not done *********
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
len of buffer now: 755924
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
644449.0


In [38]:
import numpy as np
print(buffer.reward.shape)
a = np.arange(1,5)
print(a[1:3])

(755924, 1)
[2 3]
